In [1]:
import numpy as np
import sys
sys.path.insert(0,'/local2/home/tong/caffe-master/python')
import caffe


In [2]:
caffe.set_mode_gpu()
caffe.set_device(0)


In [ ]:
#for training
solver = caffe.SGDSolver('fashion_solver3_rank.prototxt')
#solver.net.copy_from('/local2/home/tong/fashionRecommendation/data/VGG_CNN_M_2048.caffemodel')
#solver.test_nets[0].copy_from('/local2/home/tong/fashionRecommendation/data/VGG_CNN_M_2048.caffemodel')
solver.net.copy_from('/local2/home/tong/fashionRecommendation/fashion_predict3_15260.caffemodel')
solver.test_nets[0].copy_from('/local2/home/tong/fashionRecommendation/fashion_predict3_15260.caffemodel')


In [ ]:
#for saving caffemodel
net = caffe.Net('fashion_deploy3.prototxt', caffe.TEST)


In [ ]:
#verify solver.net & solver.test_nets[0]
solver.net.forward()
solver.test_nets[0].forward()

print(solver.net.blobs['accuracy'].data, 
      solver.test_nets[0].blobs['accuracy'].data)

if solver.test_nets[0].blobs['accuracy'].data > 0.3:
    print 'feasible!'

In [ ]:
#training solver.net
recordDir = '/local2/home/tong/fashionRecommendation/data/record/fashionNet3_rank/'

k = 0
for i in range (3401,100003):
    #save caffemodel every single epoch (~ 3375 iters)
    if (i % 3400 == 0):
        #copy parameters solver.net -> net
        params = net.params.keys()
        source_params = {pr: (solver.net.params[pr][0].data,solver.net.params[pr][1].data) for pr in params}
        target_params = {pr: (net.params[pr][0].data,net.params[pr][1].data) for pr in params}
        for pr in params:
            target_params[pr][0][...] = source_params[pr][0] #weights
            target_params[pr][1][...] = source_params[pr][1] #bias
        #save caffemodel
        net.save(recordDir+'fashion_predict3_rank_'+str(i)+'.caffemodel')
    # update parameters
    solver.step(1)
    # record train (accuracy, loss)
    tr_accu = open(recordDir + 'tr_accu.txt','a')
    tr_loss = open(recordDir + 'tr_loss.txt','a')
    tr_accu.write(str(i)+' '+str(solver.net.blobs['accuracy'].data)+'\r\n')
    tr_loss.write(str(i)+' '+str(solver.net.blobs['rank_Loss'].data)+'\r\n')
    tr_accu.close()
    tr_loss.close()       
    if (i % 100 == 0):
        # record test (accuracy, loss)
        te_accu = open(recordDir + 'te_accu.txt','a')
        te_loss = open(recordDir + 'te_loss.txt','a')
        te_accu.write(str(i)+' '+str(solver.test_nets[0].blobs['accuracy'].data)+'\r\n')
        te_loss.write(str(i)+' '+str(solver.test_nets[0].blobs['rank_Loss'].data)+'\r\n')
        te_accu.close()
        te_loss.close()
        # ending criteria
        if solver.test_nets[0].blobs['accuracy'].data > 0.93:
            k = k + 1
            if (k > 100):
                print '\n\ntest accuracy: {} > 0.93 for 100 times\n\n'.format(solver.test_nets[0].blobs['accuracy'].data)
                break
        

In [ ]:
#copy parameters solver.net -> net
params = net.params.keys()
source_params = {pr: (solver.net.params[pr][0].data,solver.net.params[pr][1].data) for pr in params}
target_params = {pr: (net.params[pr][0].data,net.params[pr][1].data) for pr in params}
for pr in params:
    target_params[pr][0][...] = source_params[pr][0] #weights
    target_params[pr][1][...] = source_params[pr][1] #bias


In [ ]:
#verify params copy
print 'solver.net: ', solver.net.params['metric_fc1'][0].data
print 'net:', solver.net.params['metric_fc1'][0].data


In [ ]:
#save caffemodel
net.save('ffashion_predict3_rank.caffemodel')


In [ ]:
#verify caffemodel
net_Verify = caffe.Net('fashion_deploy.prototxt',
                'fashion_predict.caffemodel',
                caffe.TEST)


In [ ]:
#verify input data (posi outfit)

#input VGG_mean.jpg
blob = caffe.proto.caffe_pb2.BlobProto()
data = open('./data/VGG_mean.binaryproto', 'rb' ).read()
blob.ParseFromString(data)
VGG_mean = np.array( caffe.io.blobproto_to_array(blob) )
VGG_mean.astype(np.uint8)

#get data from blobs
top = np.array(solver.net.blobs['top'].data[0])
bottom = np.array(solver.net.blobs['bottom'].data[0])
shoes = np.array(solver.net.blobs['shoes'].data[0])
#change data type to np.uint8
top.astype(np.uint8)
bottom.astype(np.uint8)
shoes.astype(np.uint8)
#add VGG_mean
top = top + VGG_mean[0,]
bottom = bottom + VGG_mean[0,]
shoes = shoes + VGG_mean[0,]
#BGR to RGB
top = np.array([top[2,],top[1,],top[0,]])
bottom = np.array([bottom[2,],bottom[1,],bottom[0,]])
shoes = np.array([shoes[2,],shoes[1,],shoes[0,]])
#(C,H,W) to (H,W,C)
top = np.rollaxis(top,0,3)
bottom = np.rollaxis(bottom,0,3)
shoes = np.rollaxis(shoes,0,3)
#temp buffer for using Image.fromarray(*)
temp_top = np.zeros(top.shape,np.uint8)
temp_bottom = np.zeros(bottom.shape,np.uint8)
temp_shoes = np.zeros(shoes.shape,np.uint8)
#copy data to buffer
for i in range(0,top.shape[0]):
    for j in range(0,top.shape[1]):
        for k in range(0,top.shape[2]):
            temp_top[i,j,k] = top[i,j,k]
            temp_bottom[i,j,k] = bottom[i,j,k]
            temp_shoes[i,j,k] = shoes[i,j,k]
#np.array 2 image
import Image
img_top = Image.fromarray(temp_top)
img_bottom = Image.fromarray(temp_bottom)
img_shoes = Image.fromarray(temp_shoes)
#save images
img_top.save('./top.jpg')
img_bottom.save('./bottom.jpg')
img_shoes.save('./shoes.jpg')

#verify input data (nega outfit)
#get data from blobs
top = np.array(solver.test_nets[0].blobs['top_n'].data[49])
bottom = np.array(solver.test_nets[0].blobs['bottom_n'].data[49])
shoes = np.array(solver.test_nets[0].blobs['shoes_n'].data[49])
#change data type to np.uint8
top.astype(np.uint8)
bottom.astype(np.uint8)
shoes.astype(np.uint8)
#add VGG_mean
top = top + VGG_mean
bottom = bottom + VGG_mean
shoes = shoes + VGG_mean
#BGR to RGB
top = np.array([top[2,],top[1,],top[0,]])
bottom = np.array([bottom[2,],bottom[1,],bottom[0,]])
shoes = np.array([shoes[2,],shoes[1,],shoes[0,]])
#(C,H,W) to (H,W,C)
top = np.rollaxis(top,0,3)
bottom = np.rollaxis(bottom,0,3)
shoes = np.rollaxis(shoes,0,3)
#temp buffer for using Image.fromarray(*)
temp_top = np.zeros(top.shape,np.uint8)
temp_bottom = np.zeros(bottom.shape,np.uint8)
temp_shoes = np.zeros(shoes.shape,np.uint8)
#copy data to buffer
for i in range(0,top.shape[0]):
    for j in range(0,top.shape[1]):
        for k in range(0,top.shape[2]):
            temp_top[i,j,k] = top[i,j,k]
            temp_bottom[i,j,k] = bottom[i,j,k]
            temp_shoes[i,j,k] = shoes[i,j,k]
#np.array 2 image
import Image
img_top = Image.fromarray(temp_top)
img_bottom = Image.fromarray(temp_bottom)
img_shoes = Image.fromarray(temp_shoes)
#save images
img_top.save('./top_n.jpg')
img_bottom.save('./bottom_n.jpg')
img_shoes.save('./shoes_n.jpg')